
# Generate Paths

This notebook generates path sets for a number of experiments on the PuddleWorld domain.


In [ ]:

import gym
import copy
import pickle
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from puddle_world.envs import *
from explicit_env.soln import value_iteration, q_from_v, OptimalPolicy
from unimodal_irl.utils import get_rollouts


In [ ]:

# Deterministic, Stochastic environemnts
# Wet, Dry, Any modes
# 10,000 paths for each instance

num_rollouts = 100000
rollout_batch_size = 100
num_batches = num_rollouts // rollout_batch_size
print(f"Performing {num_batches} batches of rollouts")
for transitions in ['deterministic', 'stochastic']:
    if transitions == 'deterministic':
        wind = 0.0
    else:
        wind = 0.2
    for mode in ['wet', 'dry', 'any']:
        env = CanonicalPuddleWorldEnv(mode=mode, wind=wind)
        pi = OptimalPolicy(q_from_v(value_iteration(env), env), stochastic=True)
        rollouts = []
        for batch in range(num_batches):
            print(".", end="")
            rollouts.extend(get_rollouts(env, pi, rollout_batch_size))
        print()
        
        filename = f"pw-{transitions}-{mode}.pkl"
        with open(filename, "wb") as file:
            pickle.dump(rollouts, file)
